In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_23044\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_23044\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,4
1,4,4,4
2,4,4,2
3,4,0,0
4,3,6,5


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [16]:
X = df_train[["Humidity","Moisture","Soil Type","Crop Type"]]

In [17]:
y = df_train[["Fertilizer Name"]]

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
# https://ai.plainenglish.io/the-art-of-seeing-the-unseen-deep-learning-in-rare-event-prediction-07ab3dc59194https://ai.plainenglish.io/the-art-of-seeing-the-unseen-deep-learning-in-rare-event-prediction-07ab3dc59194
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Step 1: Flatten y_train
y_train_flat = np.ravel(y_train)  # or y_train.values.ravel() if it's a DataFrame

# Step 2: Compute class weights
classes = np.unique(y_train_flat)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_flat)
class_weights_dict = dict(zip(classes, class_weights))

# Step 3: Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import numpy as np

def build_and_tune_model(X, y, base_model='random_forest', param_grid=None, cv_folds=5):
    # Flatten labels
    y_flat = np.ravel(y)

    # Compute class weights
    classes = np.unique(y_flat)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_flat)
    class_weight_dict = dict(zip(classes, class_weights))

    # Define base model
    if base_model == 'random_forest':
        model = RandomForestClassifier(class_weight=class_weight_dict, random_state=42)
        if param_grid is None:
            param_grid = {
                'classifier__n_estimators': [100, 200],
                'classifier__max_depth': [None, 10, 20],
                'classifier__min_samples_split': [2, 5],
                'classifier__max_features': ['sqrt', 'log2']
            }

    # Build pipeline
    pipeline = Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('classifier', model)
    ])

    # Cross-validation strategy
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Grid search
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf,
        scoring='f1_weighted', verbose=1, n_jobs=-1
    )
    grid_search.fit(X, y_flat)

    # Report results
    print("Best Parameters:", grid_search.best_params_)
    print(" Cross-Validated Score:", grid_search.best_score_)

    best_model = grid_search.best_estimator_
    return best_model

In [22]:
# Assuming you've got the trained best model from your tuning pipeline
best_model = build_and_tune_model(X_train, y_train)

# Evaluate on test set
evaluate_model(best_model, X_test, y_test)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters: {'classifier__max_depth': 20, 'classifier__max_features': 'sqrt', 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
 Cross-Validated Score: 0.14809591134817368


NameError: name 'evaluate_model' is not defined

In [23]:
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

In [25]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions
y_pred = best_model.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}")

# Detailed metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Test Accuracy: 0.1506

Classification Report:
              precision    recall  f1-score   support

           0       0.16      0.12      0.14     37465
           1       0.16      0.12      0.14     37584
           2       0.16      0.13      0.15     37454
           3       0.16      0.13      0.15     36590
           4       0.16      0.13      0.14     36823
           5       0.14      0.22      0.17     31462
           6       0.13      0.22      0.16     30122

    accuracy                           0.15    247500
   macro avg       0.15      0.15      0.15    247500
weighted avg       0.15      0.15      0.15    247500


Confusion Matrix:
[[4437 4084 4686 4701 4468 7427 7662]
 [4062 4393 4677 4661 4561 7359 7871]
 [4043 4160 4974 4665 4467 7317 7828]
 [3940 3963 4395 4927 4582 7120 7663]
 [4016 3985 4452 4482 4743 7354 7791]
 [3306 3265 3628 3789 3787 7055 6632]
 [3191 3178 3613 3785 3578 6033 6744]]
